In [3]:
!pip install pymysql

  Using cached PyMySQL-1.0.2-py3-none-any.whl (43 kB)


# 데이터 읽어오기

## Pandas로 Mysql 데이터 읽어오기

In [13]:
import pandas as pd
import pymysql


conn = pymysql.connect(host='localhost', user='root',
                       password='password', db='INVESTAR', charset='utf8')

### 기업 정보

In [14]:
sql="select * from company_info"
companyInfo = pd.read_sql(sql, conn)
companyInfo

,code,company,last_update
0,000020,동화약품,2021-06-09
1,000040,KR모터스,2021-06-09
2,000050,경방,2021-06-09
3,000060,메리츠화재,2021-06-09
4,000070,삼양홀딩스,2021-06-09
...,...,...,...
2446,950180,SNK,2021-06-09
2447,950190,미투젠,2021-06-09
2448,950200,소마젠,2021-06-09
2449,950210,프레스티지바이오파마,2021-06-09


### 일별 주가

In [18]:
from datetime import datetime, timedelta

yesterday = datetime.today() - timedelta(1)

sql = "SELECT * FROM daily_price WHERE code = '{}' and date >= '{}' and date <= '{}'".format(
    '000020', '2020-05-15', str(yesterday))
df = pd.read_sql(sql, conn)
df.index = df['date']
df

,code,date,open,high,low,close,diff,volume
date,,,,,,,,
2020-05-15,000020,2020-05-15,10850,10950,10450,10550,150,680362
2020-05-18,000020,2020-05-18,10550,10800,10200,10700,150,622190
2020-05-19,000020,2020-05-19,10500,10500,9910,10050,650,758107
2020-05-20,000020,2020-05-20,10100,10200,9990,10050,0,424923
2020-05-21,000020,2020-05-21,10050,10150,9710,9710,340,618577
...,...,...,...,...,...,...,...,...
2021-06-02,000020,2021-06-02,14700,14700,14450,14500,100,109559
2021-06-03,000020,2021-06-03,14550,14650,14450,14600,100,96158
2021-06-04,000020,2021-06-04,14600,14800,14550,14700,100,133900


###  MarketDB 함수 생성

일별 주가 데이터를 불러올 때마다 위의 코드를 치는 것은 귀찮으니 함수로 설정해둔다.

In [45]:
class MarketDB:
    def __init__(self):
        self.conn = pymysql.connect(
            host='localhost', user='root', password='password', db='INVESTAR', charset='utf8')
        self.codes = {}
        self.getCompanyInfo()

    def getCompanyInfo(self):
        sql = "select * from company_info"
        companyInfo = pd.read_sql(sql, self.conn)
        for idx in range(len(companyInfo)):
            self.codes[companyInfo['code'].values[idx]
                       ] = companyInfo['company'].values[idx]

    def getDailyPrice(self, code, startDate, endDate):
        codes_keys = list(self.codes.keys())
        codes_vals = list(self.codes.values())
        if code in codes_keys:
            pass
        elif code in codes_vals:
            idx = codes_vals.index(code)
            code = codes_keys[idx]
        else:
            print(f"ValueError: code({code}) doesn't exist")

        sql = "SELECT * FROM daily_price WHERE code = '{}' and date >= '{}' and date <= '{}'".format(
            code, startDate, endDate)
        df = pd.read_sql(sql, conn)
        df.index = df['date']
        return df

# 기술적 지표

## 볼린저 밴드

In [47]:
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

yesterday = datetime.today() - timedelta(1)

md = MarketDB()
df = md.getDailyPrice('현대자동차', '2017-04-13', str(yesterday))
df

,code,date,open,high,low,close,diff,volume
date,,,,,,,,
2017-04-13,005380,2017-04-13,143000,144500,141000,142500,1000,537212
2017-04-14,005380,2017-04-14,141500,143000,141000,142500,0,235481
2017-04-17,005380,2017-04-17,142500,144000,141000,144000,1500,354807
2017-04-18,005380,2017-04-18,144500,144500,141000,141000,3000,512212
2017-04-19,005380,2017-04-19,140000,140500,136500,137000,4000,1028582
...,...,...,...,...,...,...,...,...
2021-06-02,005380,2021-06-02,238500,243000,235500,238000,1000,1707167
2021-06-03,005380,2021-06-03,238500,240500,236500,238500,500,1034103
2021-06-04,005380,2021-06-04,241500,243000,238000,241500,3000,1333789
